# Исследуем API нейросетей на возможность писать корректные SQL запросы

In [96]:
import numpy as np
import pandas as pd
import psycopg2

In [97]:
DEEPSEEK_KEY = "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"

In [98]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=DEEPSEEK_KEY,
)

completion = client.chat.completions.create(
  model="deepseek/deepseek-r1:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ],
  temperature=0
)
response_message = completion.choices[0].message.content
print(response_message )

The meaning of life is a deeply personal and multifaceted concept that varies across individuals, cultures, and philosophical traditions. Here's a structured synthesis of the exploration:

1. **Philosophical Perspectives**:
   - **Existentialism**: Life has no inherent meaning; individuals must create their own purpose through choices and actions (e.g., Sartre, Camus).
   - **Absurdism**: Embraces the conflict between seeking meaning and an indifferent universe, finding joy in the struggle itself (Camus).

2. **Religious/Spiritual Views**:
   - **Theistic Religions**: Purpose often involves serving a divine being, achieving salvation, or enlightenment (e.g., Christianity, Buddhism).
   - **Eastern Philosophies**: Focus on ending suffering (Buddhism) or harmonizing with the universe (Taoism).

3. **Scientific Angle**:
   - **Biology**: Survival, reproduction, and genetic continuity, though humans seek fulfillment beyond these basics.
   - **Cosmology**: Some explore meaning through the 

Имея доступ к нейросети deepseek, распарсим таблицу данных на группы.

In [101]:
df = pd.DataFrame()
with psycopg2.connect(
    "dbname=db user=user password=passw host=127.0.0.1 port=5440"
) as conn:
    df = pd.read_sql_query("Select * FROM freelancers", conn)
df


/tmp/ipykernel_20865/3059670997.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("Select * FROM freelancers", conn)


,freelancer_id,job_category,platform,experience_level,client_region,payment_method,job_completed,earnings_usd,hourly_rate,job_success_rate,client_rating,job_duration_days,project_type,rehire_rate,marketing_spend
0,1.0,Web Development,Fiverr,Beginner,Asia,Mobile Banking,180.0,1620.0,95.79,68.73,3.18,1.0,Fixed,40.19,53.0
1,2.0,App Development,Fiverr,Beginner,Australia,Mobile Banking,218.0,9078.0,86.38,97.54,3.44,54.0,Fixed,36.53,486.0
2,3.0,Web Development,Fiverr,Beginner,UK,Crypto,27.0,3455.0,85.17,86.60,4.20,46.0,Hourly,74.05,489.0
3,4.0,Data Entry,PeoplePerHour,Intermediate,Asia,Bank Transfer,17.0,5577.0,14.37,79.93,4.47,41.0,Hourly,27.58,67.0
4,5.0,Digital Marketing,Upwork,Expert,Asia,Crypto,245.0,5898.0,99.37,57.80,5.00,41.0,Hourly,69.09,489.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,1946.0,Graphic Design,Fiverr,Beginner,USA,Mobile Banking,143.0,6823.0,22.54,75.86,4.65,13.0,Hourly,26.55,133.0
1946,1947.0,SEO,Upwork,Intermediate,Middle East,Crypto,164.0,7942.0,77.20,72.01,3.29,34.0,Hourly,24.81,343.0
1947,1948.0,SEO,PeoplePerHour,Expert,UK,Bank Transfer,236.0,9838.0,24.64,57.37,4.67,38.0,Fixed,61.52,370.0
1948,1949.0,SEO,Freelancer,Intermediate,Europe,Bank Transfer,152.0,4492.0,71.07,66.41,4.08,70.0,Fixed,32.40,32.0


Вытащим колонки и типы данных

In [102]:
columns = df.columns
numeric_types = df.select_dtypes(include=np.number).columns
non_numeric_types = df.select_dtypes(exclude=np.number).columns

(columns, numeric_types, non_numeric_types)

(Index(['freelancer_id', 'job_category', 'platform', 'experience_level',
        'client_region', 'payment_method', 'job_completed', 'earnings_usd',
        'hourly_rate', 'job_success_rate', 'client_rating', 'job_duration_days',
        'project_type', 'rehire_rate', 'marketing_spend'],
       dtype='object'),
 Index(['freelancer_id', 'job_completed', 'earnings_usd', 'hourly_rate',
        'job_success_rate', 'client_rating', 'job_duration_days', 'rehire_rate',
        'marketing_spend'],
       dtype='object'),
 Index(['job_category', 'platform', 'experience_level', 'client_region',
        'payment_method', 'project_type'],
       dtype='object'))

Из анализа датафрейма выше видно, что в текстовых колонках число уникальных данных мало (5 - 8).
Можно вытянуть их отдельно для каждой колонки. 

In [103]:
unique = {}

In [104]:
df_non_numeric_uniq = df.loc[:, non_numeric_types]
for column in df_non_numeric_uniq:
    unique[column] = list(df_non_numeric_uniq[column].unique())
unique

{'job_category': ['Web Development',
  'App Development',
  'Data Entry',
  'Digital Marketing',
  'Customer Support',
  'Content Writing',
  'Graphic Design',
  'SEO'],
 'platform': ['Fiverr', 'PeoplePerHour', 'Upwork', 'Toptal', 'Freelancer'],
 'experience_level': ['Beginner', 'Intermediate', 'Expert'],
 'client_region': ['Asia',
  'Australia',
  'UK',
  'Europe',
  'USA',
  'Middle East',
  'Canada'],
 'payment_method': ['Mobile Banking', 'Crypto', 'Bank Transfer', 'PayPal'],
 'project_type': ['Fixed', 'Hourly']}

После получения вариаций значений для колонок, получим минмумы и максимумы по колонкам числовым
Это даст нейросети больше понимания, в каких диапазонах и с какими данными она работает

In [105]:
diapazones = {}

In [106]:
df_numeric_diapazones = df.loc[:, numeric_types]
for column in df_numeric_diapazones:
    diapazones[column] = {'min': int(np.min(df_numeric_diapazones[column])),
                          			 'max': int(np.max(df_numeric_diapazones[column]))}
diapazones

{'freelancer_id': {'min': 1, 'max': 1950},
 'job_completed': {'min': 5, 'max': 299},
 'earnings_usd': {'min': 51, 'max': 9991},
 'hourly_rate': {'min': 5, 'max': 99},
 'job_success_rate': {'min': 50, 'max': 99},
 'client_rating': {'min': 3, 'max': 5},
 'job_duration_days': {'min': 1, 'max': 89},
 'rehire_rate': {'min': 10, 'max': 79},
 'marketing_spend': {'min': 0, 'max': 499}}

Думаю, что к этим данным не помешает описание. Цейросети, как и человеку, нужен контекст, с которым она будет работать.

### Dataset Features
| Column Name | Description | Data Type | Example Values |
|-------------|-------------|-----------|----------------|
| Freelancer_ID | Unique identifier for each freelancer record | integer | 1, 2, 3 |
| Job_Category | Primary classification of freelance work performed | string | Web Development, Data Entry, Content Writing |
| Platform | Freelance marketplace where work was performed | string | Fiverr, Upwork, Toptal, Freelancer, PeoplePerHour |
| Experience_Level | Freelancer's professional experience tier | string | Beginner, Intermediate, Expert |
| Client_Region | Geographical location of the client | string | Asia, Europe, USA, Canada, UK, Australia, Middle East |
| Payment_Method | Method used for financial transactions | string | Bank Transfer, PayPal, Mobile Banking, Crypto |
| Job_Completed | Number of projects successfully completed | integer | 180, 218, 27 |
| Earnings_USD | Total earnings in US Dollars | float | 1620, 9078, 3455 |
| Hourly_Rate | Freelancer's hourly compensation rate in USD | float | 95.79, 86.38, 85.17 |
| Job_Success_Rate | Percentage of successful job completions | float | 68.73, 97.54, 86.6 |
| Client_Rating | Average rating given by clients (1.0-5.0 scale) | float | 3.18, 3.44, 4.2 |
| Job_Duration_Days | Average project timeline in days | integer | 1, 54, 46 |
| Project_Type | Classification of work arrangement | string | Fixed, Hourly |
| Rehire_Rate | Percentage of clients who rehire the freelancer | float | 40.19, 36.53, 74.05 |
| Marketing_Spend | Amount invested in platform promotion in USD | integer | 53, 486, 489 |

Хорошо, что хорошие авторы kaggle дают хорошее описание своих фич) а ещё чистые данные :)

Не вижу смысла в векторном представлении этих данных по описанию и храниню их в векторной базе по типу Chroma в эмбеддингах. Запишем эти описания в поля description и подадим на вход вместе с остальным контекстом

In [107]:
feach_descriptions = {
	"Freelancer_ID":"Unique identifier for each freelancer record ",
    "Job_Category":"Primary classification of freelance work performed",
    "Platform":"Freelance marketplace where work was performed",
    "Experience_Level": "Freelancer's professional experience tier",
    "Client_Region": " Geographical location of the client ",
    "Payment_Method": "Method used for financial transactions",
    "Job_Completed": "Number of projects successfully completed",
    "Earnings_USD": "Total earnings in US Dollars",
    "Hourly_Rate ": "Freelancer's hourly compensation rate in USD",
    "Job_Success_Rate": "Percentage of successful job completions",
    "Client_Rating": "Average rating given by clients (1.0-5.0 scale)",
    "Job_Duration_Days": "Average project timeline in days",
    "Project_Type": "Classification of work arrangement",
    "Rehire_Rate": "Percentage of clients who rehire the freelancer",
    "Marketing_Spend": "Amount invested in platform promotion in USD"
}

Теперь добавим имя таблицы

In [108]:
table_name = {
    'table_name': 'freelancers',
}

СОберём из этого системный контекстный json файл

In [109]:
context = [
    table_name,
    feach_descriptions,
    unique,
    diapazones
]

In [110]:
import json
with open("context.json", 'w') as f:
	json.dump(context, f)

## Попробуем сделать запрос из тестовых вопросов с контекстом

Напишем системный промпт

In [111]:
system_prompt = {
    "role":"system",
    "content": f"""Ты - Аналитик данных и отлично знаешь PostgreSQL. Твоя задача - создать для пользователя оптмиальный SQL зпрос для СУБД PostgreSQL.
    Таблица, с которой ты работаешь, содержит следующе данные:
    {context}
	Не придумывай данные, используй только приведённую структуру. Существующие имена таблиц и переменных.
    На выход дай ТОЛЬКО SQL - запрос к таблице. Запрос выводи текстом без кавычек и без форматирования.
	"""
}
system_prompt

{'role': 'system',
 'content': 'Ты - Аналитик данных и отлично знаешь PostgreSQL. Твоя задача - создать для пользователя оптмиальный SQL зпрос для СУБД PostgreSQL.\n    Таблица, с которой ты работаешь, содержит следующе данные:\n    [{\'table_name\': \'freelancers\'}, {\'Freelancer_ID\': \'Unique identifier for each freelancer record \', \'Job_Category\': \'Primary classification of freelance work performed\', \'Platform\': \'Freelance marketplace where work was performed\', \'Experience_Level\': "Freelancer\'s professional experience tier", \'Client_Region\': \' Geographical location of the client \', \'Payment_Method\': \'Method used for financial transactions\', \'Job_Completed\': \'Number of projects successfully completed\', \'Earnings_USD\': \'Total earnings in US Dollars\', \'Hourly_Rate \': "Freelancer\'s hourly compensation rate in USD", \'Job_Success_Rate\': \'Percentage of successful job completions\', \'Client_Rating\': \'Average rating given by clients (1.0-5.0 scale)\', \

Минимальный пул вопросов, на которые предстоит ответить


	- Насколько выше доход у фрилансеров, принимающих оплату в криптовалюте, по сравнению с другими способами оплаты?  
	- Как распределяется доход фрилансеров в зависимости от региона проживания?  
	- Какой процент фрилансеров, считающих себя экспертами, выполнил менее 100 проектов?


In [130]:
# модель deepseek оказалась слишком медленной. В качестве альтернативы выбрана qwen/qwen3-30b-a3b:free
completion = client.chat.completions.create(
  model="qwen/qwen3-30b-a3b:free",
  messages=[
    system_prompt,
    {
      "role": "user",
      "content": "Насколько выше доход у фрилансеров, принимающих оплату в криптовалюте, по сравнению с другими способами оплаты? "
    }
  ],
  temperature = 0.7
)
response_message = completion.choices[0].message.content
print(response_message )

SELECT 
AVG(CASE WHEN Payment_Method = 'Crypto' THEN Earnings_USD END) AS avg_crypto_earnings,
AVG(CASE WHEN Payment_Method != 'Crypto' THEN Earnings_USD END) AS avg_non_crypto_earnings,
AVG(CASE WHEN Payment_Method = 'Crypto' THEN Earnings_USD END) - AVG(CASE WHEN Payment_Method != 'Crypto' THEN Earnings_USD END) AS earnings_difference
FROM freelancers
WHERE Payment_Method IS NOT NULL;


После получения запроса - распарсим его

In [131]:
sql_query = response_message.strip()
sql_query

"SELECT \nAVG(CASE WHEN Payment_Method = 'Crypto' THEN Earnings_USD END) AS avg_crypto_earnings,\nAVG(CASE WHEN Payment_Method != 'Crypto' THEN Earnings_USD END) AS avg_non_crypto_earnings,\nAVG(CASE WHEN Payment_Method = 'Crypto' THEN Earnings_USD END) - AVG(CASE WHEN Payment_Method != 'Crypto' THEN Earnings_USD END) AS earnings_difference\nFROM freelancers\nWHERE Payment_Method IS NOT NULL;"

In [132]:
result = pd.DataFrame()
with psycopg2.connect(
    "dbname=db user=user password=passw host=127.0.0.1 port=5440"
) as conn:
    result = pd.read_sql_query(sql_query, conn)
result


/tmp/ipykernel_20865/4133764436.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(sql_query, conn)


,avg_crypto_earnings,avg_non_crypto_earnings,earnings_difference
0,5139.301556,4973.993036,165.30852


In [133]:
# приведём результат в json
result_json = result.to_json(orient='records', force_ascii=False)

Отлично! После реглуировки температуры мы получаем примерно нужный нам результат.
Теперь нужно понять, как быть с полученными данными. Они динамические и алгоритмами будет очень трудно с ними что то делать.
Снова используем нейросеть. Но уже с другим контекстом. 

На этот раз дадим ей подумать над агрегированными данными и выдать строковый ответ.

In [ ]:
post_promt_context = {
    "role": "system",
    "content": f"""Как аналитик, посмотри на данные и сделай выводы. Вывод опиши коротко. Вывод дай без форматирования. Только по данным, которые ты видишь в таблице."""
}

In [135]:
completion = client.chat.completions.create(
  model="qwen/qwen3-30b-a3b:free",
  messages=[
    post_promt_context,
    {
      "role": "user",
      "content": result_json
    }
  ],
  temperature = 0.5
)
response_message = completion.choices[0].message.content
print(response_message )

Средние доходы от криптовалюты (5139.30) превышают средние доходы от не-криптовалютных источников (4973.99) на 165.31.


Нейросеть хорошо показала себя на первом вопросе! Теперь, попробуем остальные вопросы

Для начала - оформим все звпросы в виде функций

In [137]:
async def input_prompt(question):
	completion = client.chat.completions.create(
	model="qwen/qwen3-30b-a3b:free",
	messages=[
		system_prompt,
		{
		"role": "user",
		"content": question
		}
	],
	temperature = 0.7
	)
	return completion.choices[0].message.content

async def hidden_prompt(result_json):
	completion = client.chat.completions.create(
	model="qwen/qwen3-30b-a3b:free",
	messages=[
		post_promt_context,
		{
		"role": "user",
		"content": result_json
		}
	],
	temperature = 0.5
	)
	return completion.choices[0].message.content

async def query_result(query):
	sql_query = query.strip()
	result = pd.DataFrame()
	with psycopg2.connect(
		"dbname=db user=user password=passw host=127.0.0.1 port=5440"
	) as conn:
		result = pd.read_sql_query(sql_query, conn)
	return result

In [138]:
questions = [
    "Как распределяется доход фрилансеров в зависимости от региона проживания?",
    "Какой процент фрилансеров, считающих себя экспертами, выполнил менее 100 проектов?"
]

responses = []
for question in questions:
	query = await input_prompt(question)
	result = await query_result(query)
	response = await hidden_prompt(result.to_json(orient='records', force_ascii=False))
	responses.append({
		"question": question,
		"query": query,
		"result": result.to_dict(orient='records'),
		"response": response
	})

responses_df = pd.DataFrame(responses)
responses_df

/tmp/ipykernel_20865/907882974.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(sql_query, conn)
/tmp/ipykernel_20865/907882974.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(sql_query, conn)


,question,query,result,response
0,Как распределяется доход фрилансеров в зависим...,"SELECT Client_Region, AVG(Earnings_USD) AS Ave...","[{'client_region': 'Canada', 'average_earnings...",Средние доходы клиентов по регионам: Canada (5...
1,"Какой процент фрилансеров, считающих себя эксп...",SELECT (COUNT(*) * 100.0 / (SELECT COUNT(*) FR...,[{'percentage': 33.853354134165365}],Данные показывают значение процента в размере ...


In [139]:
responses_df['response'][0]  # выводим ответ на первый вопрос


'Средние доходы клиентов по регионам: Canada (5350) — самый высокий, затем Asia (5172), UK (5047), Australia (4966), Europe (4891), USA (4873), Middle East (4871) — самый низкий. Разница между лидером и аутсайдером составляет ~48.'

In [140]:
responses_df['response'][1]  # выводим ответ на второй вопрос

'Данные показывают значение процента в размере 33.85%.'

Попробуем попросить нейросеть составить дополнительные 5 вопросов по данной таблице

In [142]:
for_requests = {
    "role": "system",
	"content": f"Твой контекст: {context}\n Ты аналитик данных и твоя задача - ответить на вопрос пользователя коротко и по существу."
}

In [143]:
completion = client.chat.completions.create(
  model="qwen/qwen3-30b-a3b:free",
  messages=[
    for_requests,
    {
      "role": "user",
      "content": 'Придумай 5 вопросов в соответствии с контекстом. Отдели вопрос друг от друга переносами строки. Без форматирования и кавычек. Только вопросы.'
    }
  ],
  temperature = 1
)
response_message = completion.choices[0].message.content
print(response_message )

Какая профессиональная категория фрилансеров имеет самый высокий процент успешных завершений проектов  
Какие платформы демонстрируют наибольшие средние показатели заработков у фрилансеров  
Существует ли корреляция между уровнем опыта фрилансера и средним рейтингом, полученным от клиентов  
Как маркетинговые расходы влияют на количество выполненных проектов у фрилансеров  
Какие типы проектов связаны с наибольшей продолжительностью выполнения в днях


In [144]:
questions = response_message.strip().split('\n')

In [145]:
questions

['Какая профессиональная категория фрилансеров имеет самый высокий процент успешных завершений проектов  ',
 'Какие платформы демонстрируют наибольшие средние показатели заработков у фрилансеров  ',
 'Существует ли корреляция между уровнем опыта фрилансера и средним рейтингом, полученным от клиентов  ',
 'Как маркетинговые расходы влияют на количество выполненных проектов у фрилансеров  ',
 'Какие типы проектов связаны с наибольшей продолжительностью выполнения в днях']

In [146]:
for question in questions:
	query = await input_prompt(question)
	result = await query_result(query)
	response = await hidden_prompt(result.to_json(orient='records', force_ascii=False))
	responses.append({
		"question": question,
		"query": query,
		"result": result.to_dict(orient='records'),
		"response": response
	})

responses_df = pd.DataFrame(responses)

/tmp/ipykernel_20865/907882974.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(sql_query, conn)
/tmp/ipykernel_20865/907882974.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(sql_query, conn)
/tmp/ipykernel_20865/907882974.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql_query(sql_query, conn)
/tmp/ipykernel_20865/907882974.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqli

,question,query,result,response
0,Как распределяется доход фрилансеров в зависим...,"SELECT Client_Region, AVG(Earnings_USD) AS Ave...","[{'client_region': 'Canada', 'average_earnings...",Средние доходы клиентов по регионам: Canada (5...
1,"Какой процент фрилансеров, считающих себя эксп...",SELECT (COUNT(*) * 100.0 / (SELECT COUNT(*) FR...,[{'percentage': 33.853354134165365}],Данные показывают значение процента в размере ...
2,Какая профессиональная категория фрилансеров и...,"SELECT Job_Category, AVG(Job_Success_Rate) AS ...","[{'job_category': 'Graphic Design', 'avg_succe...","Средний уровень успеха в категории ""Graphic De..."
3,Какие платформы демонстрируют наибольшие средн...,"SELECT Platform, AVG(Earnings_USD) AS average_...","[{'platform': 'Fiverr', 'average_earnings': 50...","Все платформы имеют схожие средние заработки, ..."
4,Существует ли корреляция между уровнем опыта ф...,SELECT CORR(\n CASE WHEN Experience_Level =...,[{'correlation': 0.004510618109876524}],"Коэффициент корреляции равен **0.0045**, что у..."
5,Как маркетинговые расходы влияют на количество...,"SELECT corr(Marketing_Spend, Job_Completed) AS...",[{'correlation_coefficient': 0.035299152883827...,"Коэффициент корреляции (0.035) близок к нулю, ..."
6,Какие типы проектов связаны с наибольшей продо...,"SELECT Project_Type, AVG(Job_Duration_Days) AS...","[{'project_type': 'Hourly', 'average_duration'...","Средняя продолжительность проектов типа ""Hourl..."


In [147]:
responses_df.loc[:, ['question', 'response']]

,question,response
0,Как распределяется доход фрилансеров в зависим...,Средние доходы клиентов по регионам: Canada (5...
1,"Какой процент фрилансеров, считающих себя эксп...",Данные показывают значение процента в размере ...
2,Какая профессиональная категория фрилансеров и...,"Средний уровень успеха в категории ""Graphic De..."
3,Какие платформы демонстрируют наибольшие средн...,"Все платформы имеют схожие средние заработки, ..."
4,Существует ли корреляция между уровнем опыта ф...,"Коэффициент корреляции равен **0.0045**, что у..."
5,Как маркетинговые расходы влияют на количество...,"Коэффициент корреляции (0.035) близок к нулю, ..."
6,Какие типы проектов связаны с наибольшей продо...,"Средняя продолжительность проектов типа ""Hourl..."


Изучение ответов показывает, что нейросеть даёт ответы с форматированием. Учтём это в системном пре-промпте

## Оценка модели и пайплайна решений

На данный момент имеем модель, которая достаточно хорошо интерпретирует запросы пользователя в текстовый ответ.
А значит, имеем некоторый baseline. 

Пропишем собственные SQL - запросы по данным вопросам
```sql
-- Как распределяется доход фрилансеров в зависимости от региона проживания?
SELECT AVG(earnings_usd) as avg_earnings_usd, client_region as region from freelancers group by client_region order by avg_earnings_usd DESC;
-- Какой процент фрилансеров, считающих себя экспертами, выполнил менее 100 проектов?

WITH total_freelancers AS (
    SELECT COUNT(freelancer_id) AS total FROM freelancers WHERE experience_level = 'Expert'
),
filtered_freelancers AS (
    SELECT COUNT(freelancer_id) AS count FROM freelancers WHERE job_completed < 100 and experience_level = 'Expert'
)
SELECT 
    (filtered.count * 100.0) / total.total AS percent
FROM 
    filtered_freelancers filtered, total_freelancers total;

-- Какая профессиональная категория фрилансеров имеет самый высокий процент успешных завершений проектов?
SELECT AVG(Job_Success_Rate) as avg_job_rate, job_category from freelancers group by job_category order by avg_job_rate DESC;

-- Какие платформы демонстрируют наибольшие средние показатели заработков у фрилансеров?
SELECT AVG(earnings_usd) as income_avg, platform from freelancers group by platform order by income_avg DESC;

-- Существует ли корреляция между уровнем опыта фрилансера и средним рейтингом, полученным от клиентов?

with _grouped AS (
SELECT 
    CASE experience_level
        WHEN 'Intermediate' THEN 2
        WHEN 'Expert' THEN 3
        WHEN 'Beginner' THEN 1
    END as experience_level,
    client_rating
FROM freelancers)
SELECT CORR(experience_level, client_rating) as correlate FROM _grouped

-- Как маркетинговые расходы влияют на количество выполненных проектов у фрилансеров?

SELECT CORR(marketing_spend, job_completed) as correlation from freelancers


-- Какие типы проектов связаны с наибольшей продолжительностью выполнения в днях?
select AVG(job_duration_days) as avg_duration, project_type from freelancers group by project_type order by avg_duration desc;
```

Какой вывод можно сделать?  

 - Часто, нейронная сеть пишет более простые запросы (они более быстродейственны и эффективны)
 - Нейросеть умеет в семантику, в случае с корреляцией. Правильно проставляет ранги Intermediate / Beginner / Expert
 - Ни в одном запросе нейросети не было допущено логических и синтаксических ошибок
 - Выводы, которая делает нейросеть - не всегда однозначны. Это поможет решить более точное регулирование температуры в сторону нуля или сделать запоминание контекста, благодаря которому, пользователь сможет задачать уточняющие запросы по данным